In [ ]:
pip install tensorflow==2.12.0

TensorFlow is an open-source machine learning framework developed by Google. It is designed to facilitate the creation, training, and deployment of machine learning models. 

In [ ]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Imported required packages,Keras is an open-source deep learning library written in Python. It provides a high-level API that allows users to build and train neural networks with ease. Keras was designed with a focus on user-friendliness, modularity, and extensibility, making it accessible to both beginners and advanced machine learning practitioners.

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

The code you provided is using the ImageDataGenerator class from Keras, which is a powerful tool for data augmentation and preprocessing in image-based deep learning tasks. In this case, the rescale=1./255 argument is used to rescale the pixel values of the images

In [ ]:
train_generator = train_data_gen.flow_from_directory(
        "C:/Users/NITRO5/Desktop/Emotion Detection/train",
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode="categorical")

This code sets up a generator (train_generator) that will provide batches of training data. The generator will load images from the specified directory, resize them to a target size of 48x48 pixels, convert them to grayscale, and organize them into batches of size 64. The labels are assumed to be categorical, meaning they are represented as one-hot encoded vectors.

Preprocessing all train images

In [ ]:
validation_generator = validation_data_gen.flow_from_directory(
 "Desktop/Emotion Detection/test",
 target_size=(48,48),
 batch_size=64,
 color_mode="grayscale",
 class_mode="categorical"   
)

This code sets up a generator (validation_generator) that will provide batches of validation data. The generator will load images from the specified directory, resize them to a target size of 48x48 pixels, convert them to grayscale, and organize them into batches of size 64. The labels are assumed to be categorical, represented as one-hot encoded vectors.

In [ ]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

In [ ]:
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

In [ ]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation="relu"))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation="softmax"))


The ReLU function returns x if x is greater than or equal to zero, and it returns zero otherwise. Visually, this activation function "rectifies" negative values to zero, while leaving positive values unchanged.

In [ ]:
emotion_model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001,decay=1e-6),metrics=['accuracy'])

*loss="categorical_crossentropy": This specifies the loss function to be used during training. In this case, categorical_crossentropy is a commonly used loss function for multi-class classification problems with one-hot encoded target labels.

*optimizer=Adam(lr=0.0001, decay=1e-6): This specifies the optimizer to be used during training. The Adam optimizer is a popular choice that adapts the learning rate based on the gradients of the model parameters. The lr=0.0001 argument sets the initial learning rate to 0.0001, and decay=1e-6 specifies a learning rate decay to reduce the learning rate over time.

*metrics=['accuracy']: This specifies the metrics to be evaluated during training. In this case, the model's accuracy on the training data will be computed and displayed during training.

In [ ]:
emotion_model_info = emotion_model.fit_generator(
train_generator,
steps_per_epoch=28709 // 64,
epochs=50,
validation_data=validation_generator,
validation_steps=7178//64)

Building a generator for images.

In [ ]:
model_json = emotion_model.to_json()
with open("emotion_model.json","w") as json_file:
    json_file.write(model_json)

emotion_model.save_weights('emotion_model.h5')    

In [1]:
import cv2
import numpy as np
from keras.models import model_from_json

In [3]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful",3: "Happy",4: "Neutral",5:"Sad", 6:"Surprised"}
json_file = open('emotion_model.json','r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights('emotion_model.h5')
print("Loaded model ")

Loaded model 


In [ ]:
#cap = cv2.VideoCapture(0)

In [4]:
cap = cv2.VideoCapture(r"C:\Users\NITRO5\Desktop\pexels-diva-plavalaguna-6194825-3840x2160-25fps.mp4")
while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,(1280,720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier(r"C:\Users\NITRO5\Desktop\haarcascade_frontalface_default.xml")  
    gray_frame =cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = face_detector.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)
    for (x,y,w,h) in num_faces:
        cv2.rectangle(frame,(x,y-50),(x+w,y+h+10),(0,255,0),4)
        roi_grey_frame = gray_frame[y:y + h ,x:x+w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_grey_frame,(48,48)),-1),0)
        emotion_prediction= emotion_model.predict(cropped_img)
        maxindex= int(np.argmax(emotion_prediction))
        cv2.putText(frame,emotion_dict[maxindex],(x+5,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
    cv2.imshow('Emotion Detection',frame)
    if cv2.waitKey(1)& 0xFF ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 34ms/step


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


1-Opening the video file
 This line opens the video file using the cv2.VideoCapture function. The path to the video file is       specified.

2-Reading and resizing frames:
 These lines read a frame from the video using the cap.read() function and resize it to a desired       width and height using cv2.resize.
3-Face detection and emotion prediction:
  It initialize a face detector using the Haar cascade classifier XML file and convert the frame to   grayscale. The face detector is then used to detect faces in the frame using detectMultiScale, which returns the bounding box coordinates of the detected faces.
  
4-In this loop, for each detected face, a rectangle is drawn around it using cv2.rectangle. The region of interest (ROI) in the grayscale frame corresponding to the face is extracted. The ROI is then resized to the input size expected by the emotion model and passed through the model for emotion prediction. The predicted emotion is obtained by finding the index of the highest predicted value. The emotion label is then displayed on the frame using cv2.putText. 
